In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

import argparse
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected, dropout_adj
import os
import os.path as osp

from ogb.nodeproppred import PygNodePropPredDataset

import torch
from torch_geometric.data import Data

import os.path as osp

## Загрузка данных

In [3]:
data = pd.read_csv('./Data/agg_final_data.csv')

In [4]:
data.shape

(63864, 12)

In [5]:
data

,"('amount', 'mean')",client_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,9.948950e+05,1,44.0,4886.0,0,0,1,0,0,1,0.0,0.0
1,6.072894e+03,94,39.0,1510.0,1,0,0,0,0,1,0.0,0.0
2,2.787111e+05,114,60.0,2826.0,1,0,0,0,0,1,0.0,0.0
3,5.241910e+05,129,44.0,5479.0,1,0,0,0,1,0,0.0,0.0
4,4.365950e+04,166,32.0,608.0,0,0,1,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
63859,2.565470e+04,1920916,47.0,3463.0,0,0,1,0,0,1,0.0,0.0
63860,1.027485e+06,1920997,53.0,3429.0,1,0,0,0,1,0,0.0,0.0
63861,4.958946e+04,1921028,30.0,2806.0,0,0,1,0,0,1,0.0,0.0
63862,9.089023e+04,1921064,33.0,3457.0,0,1,0,0,0,1,0.0,0.0


In [6]:
unique_inn = data.client_inn.unique()

In [7]:
unique_inn

array([      1,      94,     114, ..., 1921028, 1921064, 1921084])

In [8]:
unique_inn.shape

(63864,)

# Нужно ли убирать в данных для x client_inn и partner_inn???

In [9]:
data_with_inn = data

In [10]:
data = data.drop(['client_inn'], axis=1) #, 'partner_inn'], axis=1)

In [11]:
data

,"('amount', 'mean')",years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,9.948950e+05,44.0,4886.0,0,0,1,0,0,1,0.0,0.0
1,6.072894e+03,39.0,1510.0,1,0,0,0,0,1,0.0,0.0
2,2.787111e+05,60.0,2826.0,1,0,0,0,0,1,0.0,0.0
3,5.241910e+05,44.0,5479.0,1,0,0,0,1,0,0.0,0.0
4,4.365950e+04,32.0,608.0,0,0,1,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
63859,2.565470e+04,47.0,3463.0,0,0,1,0,0,1,0.0,0.0
63860,1.027485e+06,53.0,3429.0,1,0,0,0,1,0,0.0,0.0
63861,4.958946e+04,30.0,2806.0,0,0,1,0,0,1,0.0,0.0
63862,9.089023e+04,33.0,3457.0,0,1,0,0,0,1,0.0,0.0


In [12]:
data.shape

(63864, 11)

In [13]:
data_with_inn.drop_duplicates(subset=['client_inn'])

,"('amount', 'mean')",client_inn,years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Fraud_presence,Reject_presence
0,9.948950e+05,1,44.0,4886.0,0,0,1,0,0,1,0.0,0.0
1,6.072894e+03,94,39.0,1510.0,1,0,0,0,0,1,0.0,0.0
2,2.787111e+05,114,60.0,2826.0,1,0,0,0,0,1,0.0,0.0
3,5.241910e+05,129,44.0,5479.0,1,0,0,0,1,0,0.0,0.0
4,4.365950e+04,166,32.0,608.0,0,0,1,0,0,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
63859,2.565470e+04,1920916,47.0,3463.0,0,0,1,0,0,1,0.0,0.0
63860,1.027485e+06,1920997,53.0,3429.0,1,0,0,0,1,0,0.0,0.0
63861,4.958946e+04,1921028,30.0,2806.0,0,0,1,0,0,1,0.0,0.0
63862,9.089023e+04,1921064,33.0,3457.0,0,1,0,0,0,1,0.0,0.0


## Перевод исходного датасета в датасет для графов

In [14]:
features_dataset = data.drop(['Fraud_presence'], axis=1)

In [15]:
features_dataset

,"('amount', 'mean')",years,Длина_Отн_в_днях,Клиент_Сегмент_КБ,Клиент_Сегмент_МБ,Клиент_Сегмент_МКП,Клиент_Сегмент_СБ,Клиент_НаличиеЗПпроекта_Д,Клиент_НаличиеЗПпроекта_Н,Reject_presence
0,9.948950e+05,44.0,4886.0,0,0,1,0,0,1,0.0
1,6.072894e+03,39.0,1510.0,1,0,0,0,0,1,0.0
2,2.787111e+05,60.0,2826.0,1,0,0,0,0,1,0.0
3,5.241910e+05,44.0,5479.0,1,0,0,0,1,0,0.0
4,4.365950e+04,32.0,608.0,0,0,1,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...
63859,2.565470e+04,47.0,3463.0,0,0,1,0,0,1,0.0
63860,1.027485e+06,53.0,3429.0,1,0,0,0,1,0,0.0
63861,4.958946e+04,30.0,2806.0,0,0,1,0,0,1,0.0
63862,9.089023e+04,33.0,3457.0,0,1,0,0,0,1,0.0


In [16]:
features_dataset.shape

(63864, 10)

In [17]:
x = torch.tensor(features_dataset.values.astype(np.float32)) 

In [18]:
y = torch.tensor(data['Fraud_presence'].values.astype(np.float32))

In [59]:
labelencoder = preprocessing.LabelEncoder()
data_with_inn['number_inn'] = labelencoder.fit_transform(data_with_inn['client_inn'])
data_with_inn

NameError: name 'preprocessing' is not defined

In [ ]:

>>> le.fit(data_with_inn[['client_inn']])

>>> le.transform(["tokyo", "tokyo", "paris"])
array([2, 2, 1]...)
>>> list(le.inverse_transform([2, 2, 1]))
['tokyo', 'tokyo', 'paris']

In [46]:
edge_index = data_with_inn[['client_inn']].values.T  # 'partner_inn']].values.T
edge_index = torch.tensor(edge_index).to(dtype=torch.long)

In [47]:
edge_index

tensor([[      1,      94,     114,  ..., 1921028, 1921064, 1921084]])

In [48]:
#Перевод данных в датасет для графа
data_graph = Data(x=x, y=y, edge_index=edge_index)

In [49]:
data_graph

Data(x=[63864, 10], edge_index=[1, 63864], y=[63864])

In [50]:
x = data_graph.x
N = data_graph.num_nodes

In [51]:
print(N)

63864


In [52]:
data_graph.edge_index

tensor([[      1,      94,     114,  ..., 1921028, 1921064, 1921084]])

In [53]:
data_graph.edge_index.shape

torch.Size([1, 63864])

In [54]:
path = './adj_gcn.pt'
print('Making the graph undirected.')

data_graph.edge_index, _ = dropout_adj(
       data_graph.edge_index, p=0, num_nodes=data_graph.num_nodes)

data_graph.edge_index

Making the graph undirected.


/data/home/krazheva/.conda/envs/py310/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


tensor([[      1,      94,     114,  ..., 1921028, 1921064, 1921084]])

In [55]:
data_graph

Data(x=[63864, 10], edge_index=[1, 63864], y=[63864])

In [56]:
data_graph.num_nodes

63864

In [57]:
data_graph.edge_index.shape

torch.Size([1, 63864])

In [58]:
data_graph.edge_index = to_undirected(data_graph.edge_index, num_nodes=data_graph.num_nodes)

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [28]:
print(data_graph.edge_index)

tensor([[      1,       1,       1,  ..., 1921219, 1921220, 1921222],
        [      1,     114,     129,  ..., 1448163, 1244954, 1492763]])


In [29]:
type(data_graph.edge_index)

torch.Tensor

In [30]:
torch.max(data_graph.edge_index)

tensor(1921222)

In [32]:
row, col = data_graph.edge_index
print('Computing adj...')

adj = SparseTensor(row=row, col=col, sparse_sizes=(N, N))
adj = adj.set_diag()
deg = adj.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj = deg_inv_sqrt.view(-1, 1) * adj * deg_inv_sqrt.view(1, -1)
adj = adj.to_scipy(layout='csr')

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
adj=sparse_mx_to_torch_sparse_tensor(adj)
print('Start processing')

Computing adj...
Start processing


In [33]:
num_hops = 3
#saved = torch.cat((x[train_idx], x[valid_idx], x[test_idx]), dim=0)
torch.save(data_graph, f'./graph.pt')

for i in tqdm(range(num_hops)):
    x = adj @ x
    torch.save(x, f'./graph_hope_{i+1}.pt')

100%|██████████| 3/3 [00:10<00:00,  3.60s/it]
